In [1]:
using Pkg
Pkg.add("UnROOT")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [2]:
using UnROOT

In [3]:
file = ROOTFile("data/SMHiggsToZZTo4L.root")

ROOTFile with 1 entry and 18 streamers.
data/SMHiggsToZZTo4L.root
└─ Events (TTree)
   ├─ "run"
   ├─ "luminosityBlock"
   ├─ "event"
   ├─ "⋮"
   ├─ "Electron_dzErr"
   ├─ "MET_pt"
   └─ "MET_phi"


In [4]:
events = LazyTree(file, "Events")

 Row │ Electron_mass    nElectron  luminosityBlock  nMuon   Electron_phi     M ⋯
     │ SubArray{Float3  UInt32     UInt32           UInt32  SubArray{Float3  F ⋯
─────┼──────────────────────────────────────────────────────────────────────────
 1   │ []               0          156              3       []               - ⋯
 2   │ [0.00544,        4          156              0       [0.134, -1       2 ⋯
 3   │ [-0.00609,       2          156              0       [2.18, 1.6       - ⋯
 4   │ [-0.00123]       1          156              7       [-0.643]         - ⋯
 5   │ [0.0117, 0       4          156              0       [1.01, -1.       1 ⋯
 6   │ [-0.00183]       1          156              2       [-0.497]         - ⋯
 7   │ [-0.00183]       1          156              1       [-1.47]          - ⋯
 8   │ [-0.00216]       1          156              0       [-0.633]         - ⋯
 9   │ [-0.0128,        4          156              0       [2.79, -2.       - ⋯
 10  │ []               0   

In [5]:
using AwkwardArray

In [6]:
muons = AwkwardArray.RecordArray(
        NamedTuple{(:pt, :eta, :phi, :mass, :charge, :isolation)}((
            AwkwardArray.from_iter(events.Muon_pt),
            AwkwardArray.from_iter(events.Muon_eta), 
            AwkwardArray.from_iter(events.Muon_phi), 
            AwkwardArray.from_iter(events.Muon_mass), 
            AwkwardArray.from_iter(events.Muon_charge), 
            AwkwardArray.from_iter(events.Muon_pfRelIso03_all),
        )
    ))

299973-element AwkwardArray.RecordArray{(:pt, :eta, :phi, :mass, :charge, :isolation), Tuple{AwkwardArray.ListOffsetArray{Vector{Int64}, AwkwardArray.PrimitiveArray{Float32, Vector{Float32}, :default}, :default}, AwkwardArray.ListOffsetArray{Vector{Int64}, AwkwardArray.PrimitiveArray{Float32, Vector{Float32}, :default}, :default}, AwkwardArray.ListOffsetArray{Vector{Int64}, AwkwardArray.PrimitiveArray{Float32, Vector{Float32}, :default}, :default}, AwkwardArray.ListOffsetArray{Vector{Int64}, AwkwardArray.PrimitiveArray{Float32, Vector{Float32}, :default}, :default}, AwkwardArray.ListOffsetArray{Vector{Int64}, AwkwardArray.PrimitiveArray{Int32, Vector{Int32}, :default}, :default}, AwkwardArray.ListOffsetArray{Vector{Int64}, AwkwardArray.PrimitiveArray{Float32, Vector{Float32}, :default}, :default}}, :default}:
 {pt: [63.04387f0, ..., 4.0486875f0], eta: [...], phi: [...], mass: [...], ...}
 {pt: [], eta: [], phi: [], mass: [], charge: [], isolation: []}
 {pt: [], eta: [], phi: [], mass: 

In [7]:
using PythonCall

In [8]:
four_muons = AwkwardArray.ListArray{
    AwkwardArray.Index64,
    AwkwardArray.TupleArray{Tuple{
        AwkwardArray.PrimitiveArray{Int64},
        AwkwardArray.PrimitiveArray{Int64},
        AwkwardArray.PrimitiveArray{Int64},
        AwkwardArray.PrimitiveArray{Int64}}
    }
}()

0-element AwkwardArray.ListArray{Vector{Int64}, AwkwardArray.TupleArray{NTuple{4, AwkwardArray.PrimitiveArray{Int64, Vector{Int64}, :default}}, :default}, :default}

In [9]:
push!(AwkwardArray.slot(four_muons.content, 1), 1)
push!(AwkwardArray.slot(four_muons.content, 2), 2)
push!(AwkwardArray.slot(four_muons.content, 3), 3)
push!(AwkwardArray.slot(four_muons.content, 4), 4)

1-element Vector{Int64}:
 4

In [10]:
four_muons = AwkwardArray.ListArray{
    AwkwardArray.Index64,
    AwkwardArray.PrimitiveArray{Int64},
}()

0-element AwkwardArray.ListArray{Vector{Int64}, AwkwardArray.PrimitiveArray{Int64, Vector{Int64}, :default}, :default}

In [11]:
function find_4lep(events_leptons)

    for leptons in events_leptons
        nlep = length(leptons[:charge])
        for i0 in 1:nlep
            for i1 in (i0 + 1):nlep
                if leptons[:charge][i0] + leptons[:charge][i1] != 0
                    continue
                end
                for i2 in 1:nlep
                    for i3 in (i2 + 1):nlep
                        if length(Set([i0, i1, i2, i3])) < 4
                            continue
                        end
                        if leptons[:charge][i2] + leptons[:charge][i3] != 0
                            continue
                        end
                        # push!(AwkwardArray.slot(four_muons.content, 1), (i0 - 1))  # Julia is 1-based, subtract 1 for 0-based indexing
                        # push!(AwkwardArray.slot(four_muons.content, 2), (i1 - 1))
                        # push!(AwkwardArray.slot(four_muons.content, 3), (i2 - 1))
                        # push!(AwkwardArray.slot(four_muons.content, 4), (i3 - 1))
                        
                        # AwkwardArray.end_tuple!(four_muons.content)
                        push!(four_muons.content, (i0 - 1)) # Julia is 1-based, subtract 1 for 0-based indexing
                        push!(four_muons.content, (i1 - 1))
                        push!(four_muons.content, (i2 - 1))
                        push!(four_muons.content, (i3 - 1))
                        AwkwardArray.end_list!(four_muons)
                    end
                end
            end 
        end
    end
end

find_4lep (generic function with 1 method)

In [12]:
find_4lep(muons[1:10])

In [13]:
four_muons

72-element AwkwardArray.ListArray{Vector{Int64}, AwkwardArray.PrimitiveArray{Int64, Vector{Int64}, :default}, :default}:
 [0, 1, 2, 3]
 [0, 1, 2, 6]
 [0, 1, 3, 4]
 [0, 1, 3, 5]
 [0, 1, 4, 6]
 [0, 1, 5, 6]
 [0, 2, 1, 3]
 [0, 2, 1, 6]
 [0, 2, 3, 4]
 [0, 2, 3, 5]
 [0, 2, 4, 6]
 [0, 2, 5, 6]
 [0, 4, 1, 3]
 ⋮
 [4, 6, 0, 1]
 [4, 6, 0, 2]
 [4, 6, 0, 5]
 [4, 6, 1, 3]
 [4, 6, 2, 3]
 [4, 6, 3, 5]
 [5, 6, 0, 1]
 [5, 6, 0, 2]
 [5, 6, 0, 4]
 [5, 6, 1, 3]
 [5, 6, 2, 3]
 [5, 6, 3, 4]